In [54]:
import random
from itertools import product
from typing import Union
from dataclasses import dataclass, field

import pandas as pd
import numpy as np

In [70]:
LST_FEAT = ["Part", "Ret0", "Ret1", "Tool0", "Tool1", "Tool2"]

In [55]:
@dataclass
class PartInfo:
    id: int
    n_ret: Union[int, list]
    name: str=field(init=False)
    wb: float=field(init=False)
    
    ret_name: list=field(init=False)
    ret0_wb: list=field(init=False)
    ret1_wb: list=field(init=False)
    ret2_wb: list=field(init=False)
    
    @staticmethod
    def _gen_factor(dim=2):
        return np.random.rand(dim, 2)
    
    def _gen_ret_name(self, layer, num):
        return [f'{self.name}_{layer}-{i}' for i in range(num)]
    
    def __post_init__(self):
        self.name = f'Part{self.id}'
        self.wb = self._gen_factor()
        
        self.ret_name = [self._gen_ret_name(layer, num) for layer, num in enumerate(self.n_ret)] 
        self.ret0_wb = [self._gen_factor() for _ in range(self.n_ret[0])]
        self.ret1_wb = [self._gen_factor() for _ in range(self.n_ret[1])]
        self.ret2_wb = [self._gen_factor() for _ in range(self.n_ret[2])]
        
        
@dataclass
class ToolInfo:
    id: int
    name: Union[str, None]=None
    name: str=field(init=False)
    tool0_wb: float=field(init=False)
    tool1_wb: float=field(init=False)
    tool2_wb: float=field(init=False)
    
    @staticmethod
    def _gen_factor(dim=2):
        return np.random.rand(dim, 2)
    
    def __post_init__(self):
        self.name = f'Tool_{self.id}'
        self.tool0_wb = self._gen_factor()
        self.tool1_wb = self._gen_factor()
        self.tool2_wb = self._gen_factor()

In [65]:
part_map = {i: PartInfo(i, [2,1,1]) for i in range(2)}
tool_map = [ToolInfo(i) for i in range(10)]

In [72]:
lst_comb = []
for idx, part in part_map.items():
    lst_comb.extend(product(*[[part.name], part.ret_name[0], part.ret_name[1], [tool.name for tool in tool_map[0:4]], [tool.name for tool in tool_map[4:8]], [tool.name for tool in tool_map[8:]]]))
df_data = pd.DataFrame(lst_comb, columns=LST_FEAT)

In [76]:
def cal_rn(df, dim):
    cur = df["Tool"]
    pre1 = 0
    pre2 = 0
    return cur+pre1+pre2

df_data["Tx"] = df_data[LST_FEAT].apply(cal_rn, dim=0, axis=1)
df_data["Ty"] = df_data[LST_FEAT].apply(cal_rn, dim=1, axis=1)
df_data

,Part,Ret0,Ret1,Tool0,Tool1,Tool2,Tx,Ty
0,Part0,Part0_0-0,Part0_1-0,Tool_0,Tool_4,Tool_8,0,1
1,Part0,Part0_0-0,Part0_1-0,Tool_0,Tool_4,Tool_9,0,1
2,Part0,Part0_0-0,Part0_1-0,Tool_0,Tool_5,Tool_8,0,1
3,Part0,Part0_0-0,Part0_1-0,Tool_0,Tool_5,Tool_9,0,1
4,Part0,Part0_0-0,Part0_1-0,Tool_0,Tool_6,Tool_8,0,1
...,...,...,...,...,...,...,...,...
123,Part1,Part1_0-1,Part1_1-0,Tool_3,Tool_5,Tool_9,0,1
124,Part1,Part1_0-1,Part1_1-0,Tool_3,Tool_6,Tool_8,0,1
125,Part1,Part1_0-1,Part1_1-0,Tool_3,Tool_6,Tool_9,0,1
126,Part1,Part1_0-1,Part1_1-0,Tool_3,Tool_7,Tool_8,0,1
